In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [2]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
tf.config.experimental.list_physical_devices('GPU')

[]

# **2.Preprocess data**

In [ ]:
# # Set the path to the data directory
# #Code without data augmentation
# data_dir = 'data'

# # Define the data generators
# train_datagen = ImageDataGenerator(
#     rescale=1./255,          # Normalize pixel values to be between 0 and 1
# )
# # validation_split=0.2
# val_datagen = ImageDataGenerator(rescale=1./255)
# test_datagen = ImageDataGenerator(rescale=1./255)

# # Specify the batch size and image size
# batch_size = 32
# img_size = (224, 224)

# # Create the data generators
# train_data = train_datagen.flow_from_directory(
#     os.path.join(data_dir, 'train'),
#     target_size=img_size,
#     batch_size=batch_size,
#     class_mode='categorical'
# )

# val_data = val_datagen.flow_from_directory(
#     os.path.join(data_dir, 'val'),
#     target_size=img_size,
#     batch_size=batch_size,
#     class_mode='categorical'
# )

# test_data = test_datagen.flow_from_directory(
#     os.path.join(data_dir, 'test'),
#     target_size=img_size,
#     batch_size=batch_size,
#     class_mode='categorical'
# )

In [4]:
# Set the path to the data directory
#Code with data augmentation
data_dir = 'data'

# Define the data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,          # Normalize pixel values to be between 0 and 1
    shear_range=0.2,         # Shear transformations
    zoom_range=0.2,          # Randomly zoom into images
    horizontal_flip=True,    # Randomly flip images horizontally
    rotation_range=20,       # Randomly rotate images
    width_shift_range=0.2,   # Randomly shift images horizontally
    height_shift_range=0.2,  # Randomly shift images vertically
    fill_mode='nearest'    # Strategy for filling in newly created pixels
        # Percentage of training data to use for validation
)


# train_datagen = ImageDataGenerator(
#     rescale=1./255,          # Normalize pixel values to be between 0 and 1
#     shear_range=0.2,         # Shear transformations
#     zoom_range=[0.8, 1.2],   # Randomly zoom in or out
#     horizontal_flip=True,    # Randomly flip images horizontally
#     rotation_range=20,       # Randomly rotate images
#     width_shift_range=0.2,   # Randomly shift images horizontally
#     height_shift_range=0.2,  # Randomly shift images vertically
#     fill_mode='nearest',    # Strategy for filling in newly created pixels
#     brightness_range=[0.5, 1.5],  # Adjust brightness
#     channel_shift_range=20.0,  # Randomly shift channels
#     vertical_flip=True,      # Randomly flip images vertically
#     featurewise_center=False,  # Set input mean to 0 over the dataset
#     featurewise_std_normalization=False,  # Divide inputs by std of the dataset
#     zca_whitening=False,     # Apply ZCA whitening
#     preprocessing_function=None,  # Custom preprocessing function
#     data_format='channels_last',  # Image data format
#     validation_split=0.2    # Percentage of training data to use for validation
# )
# validation_split=0.2
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Specify the batch size and image size
batch_size = 32
img_size = (224, 224)

# Create the data generators
train_data = train_datagen.flow_from_directory(
    os.path.join(data_dir, 'train'),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_data = val_datagen.flow_from_directory(
    os.path.join(data_dir, 'val'),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_data = test_datagen.flow_from_directory(
    os.path.join(data_dir, 'test'),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 6862 images belonging to 38 classes.
Found 823 images belonging to 38 classes.
Found 34 images belonging to 38 classes.


In [5]:
class_labels = []

print("Class mapping for training data:")
for class_name, class_index in sorted(train_data.class_indices.items(), key=lambda x: x[1]):
    class_labels.append(class_name)
    
output_neurons=len(class_labels) 
class_labels
    

Class mapping for training data:


['Abbott’s Babbler Malacocincla abbotti',
 'Black Bittern (Dupetor flavicollis)',
 'Blue-eared Kingfisher Alcedo meninting',
 'Blue-naped Pitta Pitta nipalensis',
 'Broad-billed Warbler Tickellia hodgsoni',
 'Cheer Pheasant (Catreus wallichii)',
 'Chestnut Munia Lonchura atricapilla',
 'Cinereous Vulture Aegypius monachus',
 'Golden Babbler Stachyris chrysaea',
 'Gould’s Shortwing Brachypteryx stellata',
 'Great Bittern Botaurus stellaris',
 'Great Hornbill (Buceros bicornis)',
 'Great Slaty Woodpecker Mulleripicus pulverulentus',
 'Ibisbill Ibidorhyncha struthersii',
 'Indian Courser Cursorius coromandelicus',
 'Indian Grassbird - Graminicola bengalensis',
 'Indian Nightjar Caprimulgus asiaticus',
 'Knob-billed Duck Sarkidiornis melanotos',
 'Northern Pintail Anas acuta',
 'Painted Stork Mycteria leucocephala',
 'Purple Cochoa Cochoa purpurea',
 'Red-headed Trogon Harpactes erythrocephalus',
 'Red-headed Vulture Sarcogyps calvus',
 'Red-necked Falcon Falco chicquera',
 'Ruby-cheeked S

# **3. Deep Model**

**3.1 Build Deep Learning Model**

In [ ]:
##Our modle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential()

model.add(Conv2D(32, (3, 3), 1, activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D())

model.add(Conv2D(64, (3, 3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3, 3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(output_neurons, activation='softmax'))  # Adjust to the number of classes (8 in this case)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
# #Model propvided by chatgpot
# # Define the model architecture
# model = models.Sequential()

# # Convolutional layers
# model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
# model.add(layers.MaxPooling2D((2, 2)))

# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))

# model.add(layers.Conv2D(128, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))

# # Flatten the output of the convolutional layers
# model.add(layers.Flatten())

# # Dense layers
# model.add(layers.Dense(128, activation='relu'))
# model.add(layers.Dense(21, activation='softmax'))  # 21 classes for bird species

# # Compile the model
# model.compile(optimizer='adam',
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])

# # Display the model summary
# model.summary()

In [ ]:
# Define the EarlyStopping callback
early_stopping = EarlyStopping(
    monitor='val_accuracy',  # Monitor validation accuracy
    patience=5,               # Allow 5 epochs with no improvement
    verbose=1,                # Print information during training
    mode='auto',              # 'auto' automatically determines the direction of improvement
    restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored quantity
)

In [ ]:

logdir = 'logs'
tf.keras.utils.enable_interactive_logging()
# Train the model with the updated architecture and callbacks
history = model.fit(train_data, epochs=200, validation_data=val_data)


## **3.2 Test**

In [ ]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_data)
print(f'Test Accuracy: {test_acc * 100:.2f}%')

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()

# Plotting every other single value, excluding the first element
plt.plot(history.history['loss'][1::2], color="teal", label='loss')

plt.plot(history.history['val_loss'][1::2], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)

plt.legend(loc="upper left")
plt.show()


In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()

# Plotting every other single value, excluding the first element
plt.plot(history.history['accuracy'], color='teal', label='accuracy')

plt.plot(history.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)

plt.legend(loc="upper left")
plt.show()


**evaluate perfomance**

In [6]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [7]:
tf.keras.utils.disable_interactive_logging()

In [ ]:

# from tensorflow.keras.models import load_model


# # Provide the path to your saved model
# model_path = 'final_86_15_accuracy_38_classes_very_important.h5'
# model = load_model(model_path)


In [27]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import logging

# Suppress TensorFlow verbose output
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Suppress other log outputs
logging.getLogger('tensorflow').setLevel(logging.ERROR)

# Assuming 'model' and 'class_labels' are already defined

# Directory path where the test images are stored
test_folder_path = 'data/test'

# Initialize a dictionary to store correct predictions count for each class
correct_predictions_count = {label: 0 for label in class_labels}
total_images_count = 0

def process_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))  # Adjust target_size based on your model's input size
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize pixel values
    return img_array

# Loop through each file and subdirectory in the directory
for item in sorted(os.listdir(test_folder_path), key=lambda x: x.lower()):
    item_path = os.path.join(test_folder_path, item)

    if os.path.isdir(item_path):
        # Handle subdirectories (if any)
        correct_predictions = 0
        total_images = 0

        # Move actual_label initialization inside the loop
        actual_label = item

        for filename in sorted(os.listdir(item_path), key=lambda x: x.lower()):
            if filename.endswith((".jpg", ".png")):
                img_path = os.path.join(item_path, filename)
                img_array = process_image(img_path)

                # Make predictions
                predictions = model.predict(img_array)

                # Get the predicted class label
                predicted_class = np.argmax(predictions)
                predicted_label = class_labels[predicted_class]

                # Check if the prediction is correct
                if predicted_label == actual_label:
                    correct_predictions += 1

                total_images += 1

        # Update the overall counts
        correct_predictions_count[actual_label] += correct_predictions
        total_images_count += total_images

        # Print the results for each class
        print(f'Class Name: {actual_label}, {correct_predictions}/{total_images}')

# Calculate and print the average accuracy
average_accuracy = sum(correct_predictions_count.values()) / total_images_count
print(f'\nAverage Accuracy: {average_accuracy:.2%}')






Class Name: Abbott’s Babbler Malacocincla abbotti, 0/1
Class Name: Black Bittern (Dupetor flavicollis), 1/1
Class Name: Blue-eared Kingfisher Alcedo meninting, 1/1
Class Name: Blue-naped Pitta Pitta nipalensis, 0/1
Class Name: Broad-billed Warbler Tickellia hodgsoni, 1/1
Class Name: Cheer Pheasant (Catreus wallichii), 1/1
Class Name: Chestnut Munia Lonchura atricapilla, 1/1
Class Name: Cinereous Vulture Aegypius monachus, 1/1
Class Name: Golden Babbler Stachyris chrysaea, 1/1
Class Name: Gould’s Shortwing Brachypteryx stellata, 1/1
Class Name: Great Bittern Botaurus stellaris, 0/1
Class Name: Great Hornbill (Buceros bicornis), 1/1
Class Name: Great Slaty Woodpecker Mulleripicus pulverulentus, 1/1
Class Name: Ibisbill Ibidorhyncha struthersii, 1/1
Class Name: Indian Courser Cursorius coromandelicus, 1/1
Class Name: Indian Grassbird - Graminicola bengalensis, 0/1
Class Name: Indian Nightjar Caprimulgus asiaticus, 0/0
Class Name: Knob-billed Duck Sarkidiornis melanotos, 1/1
Class Name: No

In [24]:
os.listdir('data/test')

['Great Slaty Woodpecker Mulleripicus pulverulentus',
 'Abbott’s Babbler Malacocincla abbotti',
 'Black Bittern (Dupetor flavicollis)',
 'Blue-eared Kingfisher Alcedo meninting',
 'Blue-naped Pitta Pitta nipalensis',
 'Broad-billed Warbler Tickellia hodgsoni',
 'Cheer Pheasant (Catreus wallichii)',
 'Chestnut Munia Lonchura atricapilla',
 'Cinereous Vulture Aegypius monachus',
 'Golden Babbler Stachyris chrysaea',
 'Gould’s Shortwing Brachypteryx stellata',
 'Great Bittern Botaurus stellaris',
 'Great Hornbill (Buceros bicornis)',
 'Ibisbill Ibidorhyncha struthersii',
 'Indian Courser Cursorius coromandelicus',
 'Indian Grassbird - Graminicola bengalensis',
 'Indian Nightjar Caprimulgus asiaticus',
 'Knob-billed Duck Sarkidiornis melanotos',
 'Northern Pintail Anas acuta',
 'Painted Stork Mycteria leucocephala',
 'Purple Cochoa Cochoa purpurea',
 'Red-headed Trogon Harpactes erythrocephalus',
 'Red-headed Vulture Sarcogyps calvus',
 'Red-necked Falcon Falco chicquera',
 'Ruby-cheeked S

In [10]:
# #Very important donot delete
# # import os
# import tensorflow as tf
# from tensorflow.keras.preprocessing import image
# import numpy as np
# import logging

# # Suppress TensorFlow verbose output
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# # Suppress other log outputs
# logging.getLogger('tensorflow').setLevel(logging.ERROR)

# # Assuming 'model' and 'class_labels' are already defined

# # Directory path where the test images are stored
# test_folder_path = 'data/test'

# # Initialize a dictionary to store correct predictions count for each class
# correct_predictions_count = {label: 0 for label in class_labels}
# total_images_count = 0

# def process_image(img_path):
#     img = image.load_img(img_path, target_size=(224, 224))  # Adjust target_size based on your model's input size
#     img_array = image.img_to_array(img)
#     img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
#     img_array = img_array / 255.0  # Normalize pixel values
#     return img_array

# # Loop through each file and subdirectory in the directory
# for item in os.listdir(test_folder_path):
#     item_path = os.path.join(test_folder_path, item)

#     if os.path.isdir(item_path):
#         # Handle subdirectories (if any)
#         correct_predictions = 0
#         total_images = 0

#         for filename in os.listdir(item_path):
#             if filename.endswith((".jpg", ".png")):
#                 img_path = os.path.join(item_path, filename)
#                 img_array = process_image(img_path)

#                 # Make predictions
#                 predictions = model.predict(img_array)

#                 # Get the predicted class label
#                 predicted_class = np.argmax(predictions)
#                 predicted_label = class_labels[predicted_class]

#                 # Check if the prediction is correct
#                 actual_label = item  # Assuming the folder name is the actual class label
#                 if predicted_label == actual_label:
#                     correct_predictions += 1

#                 total_images += 1

#         # Update the overall counts
#         correct_predictions_count[actual_label] += correct_predictions
#         total_images_count += total_images

#         # Print the results for each class
#         print(f'Class Name: {actual_label}, {correct_predictions}/{total_images}')

# # Calculate and print the average accuracy
# average_accuracy = sum(correct_predictions_count.values()) / total_images_count
# print(f'\nAverage Accuracy: {average_accuracy:.2%}')


Class Name: Great Slaty Woodpecker Mulleripicus pulverulentus, 1/1
Class Name: Abbott’s Babbler Malacocincla abbotti, 0/1
Class Name: Black Bittern (Dupetor flavicollis), 1/1
Class Name: Blue-eared Kingfisher Alcedo meninting, 1/1
Class Name: Blue-naped Pitta Pitta nipalensis, 0/1
Class Name: Broad-billed Warbler Tickellia hodgsoni, 1/1
Class Name: Cheer Pheasant (Catreus wallichii), 1/1
Class Name: Chestnut Munia Lonchura atricapilla, 1/1
Class Name: Cinereous Vulture Aegypius monachus, 1/1
Class Name: Golden Babbler Stachyris chrysaea, 1/1
Class Name: Gould’s Shortwing Brachypteryx stellata, 1/1
Class Name: Great Bittern Botaurus stellaris, 0/1
Class Name: Great Hornbill (Buceros bicornis), 1/1
Class Name: Ibisbill Ibidorhyncha struthersii, 1/1
Class Name: Indian Courser Cursorius coromandelicus, 1/1
Class Name: Indian Grassbird - Graminicola bengalensis, 0/1
Class Name: Indian Grassbird - Graminicola bengalensis, 0/0
Class Name: Knob-billed Duck Sarkidiornis melanotos, 1/1
Class Nam

In [ ]:
tf.keras.utils.enable_interactive_logging()

In [ ]:
# model.save("final_86_15_accuracy_38_classes_very_important.h5")

In [8]:
#Loading and testing the model

from tensorflow.keras.models import load_model


# # Provide the path to your saved model
model_path = 'abhishek_model_88_15_accuracy.h5'
model = load_model(model_path)

In [23]:
# Import necessary libraries and load your model
import tensorflow as tf
from PIL import Image
import numpy as np


# Load and preprocess the image
image_path = 'image.png'
img = Image.open(image_path)
img = img.resize((224, 224))  # Adjust the size based on your model's input size
img_array = np.array(img) / 255.0  # Normalize pixel values to be between 0 and 1
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Make predictions
predictions = model.predict(img_array)

# Assuming predictions is a one-hot encoded array, find the predicted class index
predicted_class_index = np.argmax(predictions)

# Print the predicted class label (replace 'class_labels' with your actual class labels)
predicted_class_label = class_labels[predicted_class_index]
print("Predicted Class Label:", predicted_class_label)


Predicted Class Label: Abbott’s Babbler Malacocincla abbotti


In [ ]:
+-------------------------------------------------------------------------------------# model.save("19mildrop0_2.h5")